# Test fit of a single object

Here we look at a single object in detail.

The notebook can be downloaded [here](https://github.com/lephare-photoz/lephare/blob/main/docs/notebooks/Testing_fit_of_one_object_of_the_catalogue.ipynb).

In [ ]:
import os
import lephare as lp
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

In [ ]:
config = lp.default_cosmos_config.copy()
config.update(
    {
        # "ZPHOTLIB": "VISTA_COSMOS_FREE,ALLSTAR_COSMOS,QSO_COSMOS",
        "ZPHOTLIB": "BC03_COSMOS,ALLSTAR_COSMOS,QSO_COSMOS",
        "CAT_IN": f"{lp.LEPHAREDIR}/examples/COSMOS.in",
        "CAT_OUT": "zphot_short.out",
        "ADD_EMLINES": "0,100",
        "AUTO_ADAPT": "NO",
        "Z_STEP": "0.02,0,1",  # We are looking at a source at z<1
        "CAT_LINES": "1,100",
        "SPEC_OUT": "NO",
        "PARA_OUT": f"{lp.LEPHAREDIR}/examples/output.para",
        "VERBOSE": "NO",
        "ZFIX": "NO",
        "SEL_AGE": "$LEPHAREDIR/sed/GAL/BC03_CHAB/AGE_BC03COMB.dat",
    }
)

In [ ]:
# Get the auxiliary files required.
lp.data_retrieval.get_auxiliary_data(
    keymap=config,
    additional_files=[
        "examples/COSMOS.in",
        "sed/GAL/BC03_CHAB/BC03COMB_MOD.list",
        "sed/GAL/BC03_CHAB/AGE_BC03COMB.dat",
    ],
)
files = []
with open(os.path.join(os.environ["LEPHAREDIR"], "sed/GAL/BC03_CHAB/BC03COMB_MOD.list")) as f:
    for ll in f.readlines():
        name = ll.split()[0]
        files.append(os.path.join("sed/GAL", name))
lp.data_retrieval.get_auxiliary_data(keymap=config, additional_files=files)

In [ ]:
# we can write the config to a file to keep a record
config_file = "./config_file.para"
lp.write_para_config(config, config_file)
# before running PhotoZ we must run filt, sedtolib and maggal
filterLib = lp.Filter(config_file=config_file)
filterLib.run()
# Note how native c++ classes require the config in the lephare native format
keymap = lp.all_types_to_keymap(config)
sedlib = lp.Sedtolib(config_keymap=keymap)
sedlib.run(typ="STAR", star_sed="$LEPHAREDIR/sed/STAR/STAR_MOD_ALL.list")
sedlib.run(typ="QSO", qso_sed="$LEPHAREDIR/sed/QSO/SALVATO09/AGN_MOD.list", gal_lib="LIB_QSO")
# sedlib.run(typ="GAL", gal_sed="$LEPHAREDIR/sed/GAL/COSMOS_SED/COSMOS_MOD.list", gal_lib="LIB_GAL")
sedlib.run(
    typ="GAL",
    gal_sed="$LEPHAREDIR/sed/GAL/BC03_CHAB/BC03COMB_MOD.list",
    gal_lib="LIB_GAL",
    sel_age="$LEPHAREDIR/sed/GAL/BC03_CHAB/AGE_BC03COMB.dat",  # Seems not read here. Why?
)

maglib = lp.MagGal(config_keymap=keymap)
maglib.run(typ="STAR", lib_ascii="NO", star_lib_out="ALLSTAR_COSMOS")
maglib.run(
    typ="QSO",
    lib_ascii="NO",
    mod_extinc="0,1000",
    eb_v="0.,0.1,0.2,0.3",
    extinc_law="SB_calzetti.dat",
    qso_lib_in="LIB_QSO",
)
maglib.run(
    typ="GAL",
    lib_ascii="NO",
    gal_lib_in="LIB_GAL",
    # gal_lib_out="VISTA_COSMOS_FREE",
    # mod_extinc="18,26,26,33,26,33,26,33",
    # extinc_law="SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat",
    # em_lines="EMP_UV",
    # em_dispersion="0.5,0.75,1.,1.5,2.",
    gal_lib_out="BC03_COSMOS",
    eb_v="0.,0.1,0.2,0.3,0.4,0.5",
    mod_extinc="0,12",
    extinc_law="SB_calzetti.dat",
    em_lines="PHYS",
    em_dispersion="1.",
)

In [ ]:
photz = lp.PhotoZ(keymap)

In [ ]:
filts = lp.filterSvc.FilterSvc.from_config(config_file)

In [ ]:
# for filt in filts:
#    plt.figure()
#    filt.plot_filter_curve()

In [ ]:
cat = np.loadtxt(f"{lp.LEPHAREDIR}/examples/COSMOS.in")
nfilt = 30
pos = 57
entry = cat[pos]
identifier = str(entry[0])
fluxes = entry[np.arange(1, nfilt * 2 + 1, 2)]
efluxes = entry[np.arange(1, nfilt * 2 + 1, 2) + 1]
context = int(entry[-3])
zspec = entry[-2]
additional_str = str(entry[-1])

In [ ]:
src = lp.onesource(0, photz.gridz)
src.readsource(identifier, fluxes, efluxes, context, zspec, additional_str)

In [ ]:
photz.prep_data(src)

In [ ]:
a0 = photz.compute_offsets([])
src.adapt_mag(a0);

In [ ]:
photz.fit_onesource(src)
print(src.zmin, src.chimin)

In [ ]:
photz.uncertainties_onesource(src)

In [ ]:
photz.physpara_onesource(src)

In [ ]:
src.spec, src.indmin, src.consiz

In [ ]:
photz.writeSpec_onesource(src, ".")
lp._spec.plotspec("./Id" + src.spec + ".spec")
os.remove("./Id" + src.spec + ".spec")